In [21]:
import os
import uuid
import numpy as np
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
from pydantic import BaseModel

from dotenv import load_dotenv
load_dotenv(override=True)

import warnings
warnings.filterwarnings("ignore")

In [22]:
class ImageMeta(BaseModel):
    """ Schema ของภาพที่จำเป็นต้อง embed เพื่อใช้ในการ Retrieve ข้อมูล ด้วย metadata """
    filename: str
    project: str
    desc: str

In [23]:
class ImageEmbedder:
    def __init__(self, client: QdrantClient, collection_name: str, model_name: str = "clip-ViT-B-32"):
        self.client = client
        self.collection_name = collection_name
        self.client.recreate_collection(
            collection_name=self.collection_name,
            vectors_config=VectorParams(size=512, distance=Distance.COSINE)
        )
        self.model = SentenceTransformer(model_name)

    def _embed_image(self, image_path: str) -> np.ndarray:
        img = Image.open(image_path).convert("RGB")
        emb = self.model.encode([img], convert_to_tensor=True)[0].cpu().numpy()
        return emb

    def _set_point(self, vec: np.ndarray, image_path: str, project: str, desc: str) -> PointStruct:
        return PointStruct(
            id=str(uuid.uuid4()), 
            vector=vec.tolist(), 
            payload=ImageMeta(filename=image_path, project=project,desc=desc).dict()
        )

    def add_image(self, image_path: str, project: str, desc: str):
        vec = self._embed_image(image_path)
        point = self._set_point(vec, image_path, project, desc)

        self.client.upsert(
            collection_name=self.collection_name,
            points=[point]
        )

In [24]:
# Create an instance of QdrantClient and ImageEmbedder
QDRANT_URL = os.getenv("QDRANT_URL")

qdrant_client = QdrantClient(url=QDRANT_URL)
embedder = ImageEmbedder(client=qdrant_client, collection_name="cm-images", model_name="clip-ViT-B-32")

In [25]:
# Embed and add images
embedder.add_image("images/site_a.jpg", "A", "ภาพไซต์งานก่อสร้าง A ที่มีการติดตั้งโครงสร้างพื้นฐาน บริเวณกลางเมือง")
embedder.add_image("images/blueprint.jpg", "B", "blueprint อาคาร B ที่มีการออกแบบโครงสร้างพื้นฐาน บริเวณกลางเมือง")

# Text-to-image search

In [26]:
# ควรจะต้องลองใช้ hybrid search นะ

In [27]:
# ใส่ข้อความที่ต้องการค้นหา
query_text = " blueprint "

In [28]:
# สร้างเวกเตอร์จากข้อความ ด้วย CLIP
query_vec = embedder.model.encode([query_text], convert_to_tensor=True)[0].cpu().numpy()

In [29]:
# ค้นหาใน Qdrant (top-3 ที่ใกล้สุด)
hits = qdrant_client.search(
    collection_name="images",
    query_vector=query_vec.tolist(),
    limit=1
)

In [30]:
print("ผลลัพธ์:")
for hit in hits:
    print(f"ไฟล์: {hit.payload['filename']} | รายละเอียด: {hit.payload['desc']} | Project: {hit.payload['project']}")

ผลลัพธ์:
ไฟล์: images/blueprint.jpg | รายละเอียด: blueprint อาคาร B | Project: B
